# Unity ML Agents
## Imitation Learning Walkthrough

In [ ]:
import os
from ppo.models import *
from ppo.trainer import ImitationTrainer
from unityagents import UnityEnvironment

### Hyperparameters

In [ ]:
### General parameters
max_steps = 5e5 # Set maximum number of steps to run environment.
run_path = "imitation-6" # The sub-directory name for model and summary statistics
load_model = False # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 50 # Frequency at which to save training statistics.
save_freq = 500 # Frequency at which to save model.
env_name = "TennisImitation" # Name of the training environment file.
keep_checkpoints = 5

### Algorithm-specific parameters for tuning
beta = 1e-3 # Strength of entropy regularization
buffer_size = 10 # How large the experience buffer should be before gradient descent.
learning_rate = 3e-4 # Model learning rate.
hidden_units = 64 # Number of units in hidden layer.
batch_size = 32 # How many experiences per gradient descent update step.

### Load the environment

In [ ]:
env = UnityEnvironment(file_name=env_name)
print(str(env))
player_brain = env.brain_names[1]
agent_brain = env.brain_names[0]

### Train the Agent(s)

In [ ]:
model_path = './models/{}'.format(str(run_path))
summary_path = './summaries/{}'.format(str(run_path))

tf.reset_default_graph()

# Create the Tensorflow model graph
imi_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units,
                               beta=beta, mode="imitation")

is_continuous = (env.brains[player_brain].action_space_type == "continuous")
use_observations = (env.brains[player_brain].number_observations > 0)
use_states = (env.brains[player_brain].state_space_size > 0)



if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver(max_to_keep=keep_checkpoints)

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps = sess.run(imi_model.global_step)
    summary_writer = tf.summary.FileWriter(summary_path)
    infos = env.reset(train_mode=False)
    player_info = infos[player_brain]
    agent_info = infos[agent_brain]
    trainer = ImitationTrainer(imi_model, sess, agent_info, is_continuous, use_observations, use_states)
    while steps <= max_steps or not train_model:
        if env.global_done:
            infos = env.reset(train_mode=False)
            player_info = infos[player_brain]
            agent_info = infos[agent_brain]
        # Decide and take an action
        new_info = trainer.take_action(player_info, agent_info, env, player_brain, agent_brain)
        infos = new_info
        player_info = infos[player_brain]
        agent_info = infos[agent_brain]
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(imi_model.increment_step)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [ ]:
export_graph(model_path, env_name)